# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
F. Xu  ->  F. Xu  |  ['F. Xu']


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 104 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.10601


extracting tarball to tmp_2509.10601...

 done.


/tmp/ipykernel_3389/2822249172.py:52: LatexWarning: 2509.10601 did not run properly
expected string or bytes-like object
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2509.12058/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors' from 'tmp_2509.12058/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'affils' from 'tmp_2509.12058/affils.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure RGB_Outflows_v5.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\linewidth]{RGB_Outflows_v5.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure ngc4424_MW_figures_v2.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\linewidth]{ngc4424_MW_figures_v2.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Pyt

Found 52 bibliographic references in tmp_2509.12058/main.bbl.
Issues with the citations
syntax error in line 256: '=' expected
Retrieving document from  https://arxiv.org/e-print/2509.12067


extracting tarball to tmp_2509.12067... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.12058-b31b1b.svg)](https://arxiv.org/abs/2509.12058) | **Characterization of Two Cool Galaxy Outflow Candidates Using Mid-Infrared Emission from Polycyclic Aromatic Hydrocarbons**  |
|| J. Sutter, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-09-16*|
|*Comments*| *Accepted for publication in The Astrophysical Journal Letters, 12 pages, 4 figures and 1 table*|
|**Abstract**|            We characterize two candidate cool galactic outflows in two relatively low mass, highly inclined Virgo cluster galaxies: NGC4424 and NGC4694. Previous analyses of observations using the Atacama Large Millimeter Array (ALMA) carbon monoxide (CO) line emission maps did not classify these sources as cool outflow hosts. Using new high sensitivity, high spatial resolution, JWST mid-infrared photometry in the polycyclic aromatic hydrocarbon (PAH)-tracing F770W band, we identify extended structures present off of the stellar disk. The identified structures are bright in the MIRI F770W and F2100W bands, suggesting they include PAHs as well as other dust grains. As PAHs have been shown to be destroyed in hot, ionized gas, these structures are likely to be outflows of cool (T$\leq 10^4$K) gas. This work represents an exciting possibility for using mid infrared observations to identify and measure outflows in lower mass, lower star formation galaxies.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.10665-b31b1b.svg)](https://arxiv.org/abs/2509.10665) | **SDSS-V Local Volume Mapper (LVM): Revealing the Structure of the Rosette Nebula**  |
|| M. A. Villa-Durango, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2025-09-16*|
|*Comments*| *Accepted for publication in MNRAS*|
|**Abstract**|            The Rosette Nebula is a well-known H II region shaped by the interaction of gas with the OB stars of the NGC 2244 stellar association. Located within the remnant of a giant molecular cloud, it exhibits a complex structure of ionized gas, molecular material, dust, and embedded clusters. In October 2023, the region was observed as part of the SDSS-V Local Volume Mapper (LVM) integral field spectroscopy survey. Covering a radius of approximately 1 degree, the dataset comprises 33,326 spectra with spatially resolved information spanning 390 - 980 nm. We present a structural analysis of the ionized, molecular, and dusty components using multi-wavelength observations: optical spectroscopy from SDSS-V LVM, 12CO emission from PMO/MWISP (sub-millimeter), and dust emission from WISE (12 micron) and Herschel (far-infrared). These datasets were complemented with the positions of ionizing stars to study emission structures traced by H alpha, H beta, [O III], [N II], and [S II], as well as the spatial distribution of line ratios (H alpha/H beta, [O III]/H beta, [N II]/H alpha, and [S II]/H alpha) relative to the surrounding molecular cloud. Our analysis reveals interaction zones between ionized and neutral gas, including filaments, globules, and dense regions with or without ongoing star formation. Radial and quadrant-based flux profiles further highlight morphological and ionization variations, supporting the scenario in which the Rosette Nebula evolved from a non-homogeneous molecular cloud with a thin, sheet-like structure.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.10836-b31b1b.svg)](https://arxiv.org/abs/2509.10836) | **Measuring neutrino masses with joint JWST and DESI DR2 data**  |
|| S.-H. Zhou, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-09-16*|
|*Comments*| *13 pages, 4 figures*|
|**Abstract**|            Early JWST observations reveal an unexpectedly abundant population of high-redshift candidate massive galaxies at $z \gtrsim 7$, and recent DESI measurements show a preference for dynamical dark energy, which together present a significant challenge to the standard $\Lambda$CDM cosmology. In this work, we jointly analyze high-redshift galaxy data from JWST, baryon acoustic oscillations data from DESI DR2, and cosmic microwave background (CMB) data from Planck and ACT, measuring the total neutrino mass $\sum m_{\nu}$. We consider three dark energy models ($\Lambda$CDM, $w$CDM, and $w_0w_a$CDM) and three mass hierarchies. Our results indicate that in the $w_0w_a$CDM model, adding JWST data to CMB+DESI tightens the upper limit of $\sum m_{\nu}$ by about $5.8\%-10.2\%$, and we obtain $\sum m_{\nu} < 0.167~\mathrm{eV}$ ($2\sigma$) in the normal hierarchy (NH) case. Furthermore, JWST also offers indicative lower limits on star formation efficiency parameter of $f_{*,10} \gtrsim 0.146-0.161$. Bayesian evidence weakly favors the $w_0w_a$CDM+$\sum m_{\nu}$(NH) model relative to the $\Lambda$CDM+$\sum m_{\nu}$(NH) model using CMB+DESI+JWST data. These results suggest that the joint analysis of high-redshift JWST data and low-redshift DESI data provides compelling constraints on neutrino mass and merits further investigation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.11029-b31b1b.svg)](https://arxiv.org/abs/2509.11029) | **The M- and X-class White-light Flares in Super Active Region NOAA 13664/13697 Observed by ASO-S/LST/WST**  |
|| Z. Jing, Y. Li, <mark>J. Li</mark>, Q. Li |
|*Appeared on*| *2025-09-16*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**|            Solar white-light flares (WLFs) have been observed since 1859, but their occurrence rate is not yet fully understood. The physical properties of WLFs in super active regions (SARs) are also well worth investigating. With full-disk images at 3600 Å (in the Balmer continuum) from the White-light Solar Telescope (WST) on board the Advanced Space-based Solar Observatory, we here study the M- and X-class WLFs occurring in SAR NOAA 13664/13697 (a same region in two solar Carrington rotations) during May/June 2024. 48 WLFs at 3600 Å are identified from 89 available samples with an occurrence rate of 53.9%, which is much higher than that (23.9%) of a longterm-continuous data set from October 2022 to May 2023 in our previous work (Jing et al. 2024). In particular, with an additional sample of over 730 M- and X-class flares from October 2022 to June 2024, we find that the occurrence rate of WLFs shows a good correlation with the solar cycle represented by sunspot counts. As regards the properties of the emission at 3600 Å, the WLFs in SAR NOAA 13664/13697 have some different characteristics, say, a longer duration but a weaker relative enhancement and a smaller brightening area compared with the previous long-term-continuous sample. We also find that for WLFs in NOAA 13664/13697 the relation between the duration and energy at 3600 Å is described by a power-law with index of 0.35, which is similar to the results found for superflares in Sun-like stars (Kowalski 2024). All these help us understand the solar WLFs in SARs and also provide important insights into the superflares on Sun-like stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.11305-b31b1b.svg)](https://arxiv.org/abs/2509.11305) | **The SRG/eROSITA All-Sky Survey $J$-Band Follow-Up Observations for Selected High-Redshift Galaxy Cluster Candidates**  |
|| N. Zimmermann, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-09-16*|
|*Comments*| *24 pages, 13 figures, 8 tables, submitted to A&A*|
|**Abstract**|            We select galaxy cluster candidates from the high-redshift (BEST_Z > 0.9) end of the first SRG/eROSITA All-Sky Survey (eRASS1) galaxy cluster catalogue, for which we obtain moderately deep J-band imaging data with the OMEGA2000 camera at the 3.5m telescope of the Calar Alto Observatory. We include J-band data of four additional targets obtained with the three-channel camera at the 2m Fraunhofer telescope at the Wendelstein Observatory. We complement the new J-band photometric catalogue with forced photometry in the i- and z-bands of the tenth data release of the Legacy Survey (LSDR10) to derive the radial colour distribution around the eRASS1 clusters. Without assuming a priori to find a cluster red sequence at a specific colour, we try to find a radially weighted colour over-density to confirm the presence of high-redshift optical counterparts for the X-ray emission. We compare our confirmation with optical properties derived in earlier works based on LSDR10 data to refine the existing high-redshift cluster confirmation of eROSITA-selected clusters. We attempt to calibrate the colour-redshift-relation including the new J-band data by comparing our obtained photometric redshift estimate with the spectroscopic redshift of a confirmed, optically selected, high-redshift galaxy cluster. We confirm 9 out of 18 of the selected galaxy cluster candidates with a radial over-density of similar coloured galaxies for which we provide a photometric redshift estimate. We can report an increase in the relative colour measurement precision from 8% to 4% when including J-band data. In conclusion, our findings indicate a not insignificant spurious contaminant fraction at the high-redshift end (BEST_Z > 0.9) of the eROSITA/eRASS1 galaxy cluster catalogue, as well as it underlines the necessity for wide and deep near infrared imaging data for confirmation and characterisation of high-$z$ galaxy clusters.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.11475-b31b1b.svg)](https://arxiv.org/abs/2509.11475) | **Dense Molecular Ring-like structure in gaseous CO depletion region G34.74-0.12**  |
|| S. Lin, et al. -- incl., <mark>F. Xu</mark> |
|*Appeared on*| *2025-09-16*|
|*Comments*| *16 pages, 8 figures, Accepted for publication in ApJL*|
|**Abstract**|            We report the discovery of a dense molecular ring-like structure in a dense (10$^5$ cm$^{-3}$), cold (pc-scale CO depletion at a factor of 5), and young (10$^4$ year) star-forming region G34.74-0.12, revealed by C$^{18}$O (2-1), HNC (1-0), and N$_2$H$^+$ (1-0) observations with the Atacama Large Millimeter/submillimeter Array (ALMA). The ring-like structure is redshifted with respect to the clump, spanning from $V_{\rm sys,lsr} + 0.9$ to $V_{\rm sys,lsr} + 2.9$ km s$^{-1}$, with a total mass of 109 $M_{\odot}$. It is spatially coincident with 1.3 mm and 3.0 mm dust continuum emission from cores, and several protostellar outflows. However, no free-free emission or H\textsc{ii} region is detected in association with this structure. With a slow expansion speed indicated by the position-velocity diagram, this ring structure differs from rings previously identified in more evolved star-forming regions. Possible explanations for the ring-like structure include a relic wind-blown bubble produced by a deeply embedded young stellar object, a hollow cavity formed by cloud-cloud interactions, a gas ring resulting from a temperature gradient, or a line-of-sight superposition of multiple outflows or dense clouds. This discovery offers a rare observational glimpse into the earliest dynamical processes involved in massive star formation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.12067-b31b1b.svg)](https://arxiv.org/abs/2509.12067) | **Observational Evidence of Dust Evolution in Supernova Remnants: Size Redistribution Toward Larger Grains in Early Sedov Phase**  |
|| H. Zhao, B. Chen, <mark>J. Li</mark> |
|*Appeared on*| *2025-09-16*|
|*Comments*| *12 pages, 1 table, 3 figures, accepted for publication in ApJL*|
|**Abstract**|            Recent observations have revealed that dust is widespread and abundant in galaxies up to $z\,{\backsimeq}\,8$, significantly influencing their appearance and spectral properties. In the early Universe, dust is thought to form primarily in supernova (SN) ejecta, but also undergoes destruction by the reverse shock. Studying dust in local supernova remnants (SNRs) of different sizes and ages thus provides key constraints on dust formation and evolution during cosmic dawn. Using the newly released 3D $R_{\rm V}$ map, we derived local $R_{\rm V} \equiv A_{\rm V}/E(B-V)$ values for dust in 14 Galactic SNRs in the early Sedov phase and their surrounding interstellar medium (ISM). For the first time, we detect a moderately strong positive correlation between the difference in SNR and ISM $R_{\rm V}$ ($\Delta R_{\rm V}$) and SNR radius ($R_{\rm SNR}$), with a Spearman coefficient of $r_{\rm s}\,{=}\,0.62\,{\pm}\,0.14$. This trend offers direct observational evidence for a redistribution of dust grain sizes toward larger grains during SN shock processing, consistent with theoretical models. Our findings provide essential observational constraints on dust size evolution in SNRs and important implications for understanding the rapid enrichment and survival of dust in the early Universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.10601-b31b1b.svg)](https://arxiv.org/abs/2509.10601) | **Active galactic nuclei do not exhibit stably periodic brightness variations**  |
|| <mark>K. El-Badry</mark>, D. W. Hogg, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-09-16*|
|*Comments*| *16 pages, 3 figures, submitted to OJAp*|
|**Abstract**|            Periodic variability in active galactic nuclei (AGN) light curves has been proposed as a signature of close supermassive black hole (SMBH) binaries. Recently, 181 candidate SMBH binaries were identified in Gaia DR3 based on periodicity in their $\sim$1000-day light curves. By supplementing Gaia photometry with longer-baseline light curves from the Zwicky Transient Facility (ZTF) and the Catalina Real Time Transient Survey (CRTS), we test whether the reported periodic signals persist beyond the Gaia DR3 time window. We find that in all 116 cases with available ZTF data, the Gaia-inferred periodic model fails to predict subsequent variability, which appears stochastic rather than periodic. The periodic candidates thus overwhelmingly appear to be false positives; red noise contamination appears to be the primary source of false detections. We conclude that truly periodic AGN are exceedingly rare, with at most a few in $10^6$ AGN exhibiting stable periodicity on 100 to 1000 day timescales. Models predict that the Gaia AGN light curve sample should contain dozens of true SMBH binaries with periods within the observational baseline, so the lack of strictly periodic light curves in the sample suggests that most short-period binary AGN do not have light curves dominated by simple periodicity.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error expected string or bytes-like object</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures []
exported in  _build/html/2509.12058.md


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Whitman}{Whitman College, 345 Boyer Avenue, Walla Walla, WA 99362, USA}$
$\newcommand{\Ox}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\UGent}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\STScI}{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\MPIA}{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\AURA}{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\STScIESA}{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\UCSD}{Department of Astronomy \& Astrophysics, University of California, San Diego, 9500 Gilman Dr., La Jolla, CA 92093, USA}$
$\newcommand{\JHU}{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\OSU}{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, OH 43210, USA}$
$\newcommand{ÇAPP}{Center for Cosmology and Astroparticle Physics (CCAPP), 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\ARI}{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstr. 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\UCT}{Department of Astronomy, University of Cape Town, Rondebosch 7701, South Africa}$
$\newcommand{\UConn}{Department of Physics, University of Connecticut, 196A Auditorium Road, Storrs, CT 06269, USA}$
$\newcommand{\UHawaii}{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UniCA}{Université Côte d'Azur, Observatoire de la Côte d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}$
$\newcommand{\UAlberta}{Dept. of Physics, University of Alberta, 4-183 CCIS, Edmonton, Alberta, T6G 2E1, Canada}$
$\newcommand{\Arcetri}{INAF — Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Florence, Italy}$
$\newcommand{\UWyoming}{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LJMU}{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\newcommand{\ITA}{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\CfA}{Center for Astrophysics \mid Harvard \& Smithsonian, 60 Garden St., 02138 Cambridge, MA, USA}$
$\newcommand{\MPE}{Max-Planck-Institut für Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\newcommand{\Surrey}{Department of Physics, University of Surrey, Guildford GU2 7XH, UK}$
$\newcommand{\ESO}{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\IWR}{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\ulyon}{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\newcommand{\COOL}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\OAN}{Observatorio Astron{ó}mico Nacional (IGN), C/ Alfonso XII 3, E-28014 Madrid, Spain}$
$\newcommand{\UBonn}{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\kipac}{Kavli Institute for Particle Astrophysics \& Cosmology (KIPAC), Stanford University, CA 94305, USA}$
$\newcommand{\Umanc}{Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, University of Manchester, Oxford Road, Manchester M13 9PL, UK}$
$\newcommand{\NRAO}{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}$
$\newcommand{\ANU}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\AThreeD}{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\newcommand{\HD}{\label{HD} Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\IAC}{Instituto de Astrofísica de Canarias, C/ Vía Láctea s/n, E-38205, La Laguna, Spain}$
$\newcommand{\ULL}{Departamento de Astrofísica, Universidad de La Laguna, Av. del Astrofísico Francisco Sánchez s/n, E-38206, La Laguna, Spain}$
$\newcommand{\Princeton}{Department of Astrophysical Sciences, Princeton University, 4 Ivy Lane, Princeton, NJ 08544, USA}$
$\newcommand{\IRAM}{IRAM, 300 rue de la Piscine, 38400 Saint Martin d'Héres, France}$
$\newcommand{\LERMA}{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universités, 75014 Paris}$
$\newcommand{\YB}{Centro de Desarrollos Tecnológicos, Observatorio de Yebes (IGN), 19141 Yebes, Guadalajara, Spain}$
$\newcommand{\uwa}{International Centre for Radio Astronomy Research, University of Western Australia, 7 Fairway, Crawley, 6009, WA, Australia}$
$\newcommand{\UMD}{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\NOIRLab}{International Gemini Observatory/NSF NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA}$
$\newcommand\CII{[C{\small{II}}]}$
$\newcommand\NII{[N{\small{II}}]}$
$\newcommand\OI{[O{\small{I}}]}$
$\newcommand\Rpah{R_{\rm{PAH}}}$
$\newcommand\micron{\mum}$</div>



<div id="title">

# Characterization of Two Cool Galaxy Outflow Candidates Using Mid-Infrared Emission from Polycyclic Aromatic Hydrocarbons

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.12058-b31b1b.svg)](https://arxiv.org/abs/2509.12058)<mark>Appeared on: 2025-09-16</mark> -  _Accepted for publication in The Astrophysical Journal Letters, 12 pages, 4 figures and 1 table_

</div>
<div id="authors">

J. Sutter, et al.

</div>
<div id="abstract">

**Abstract:** We characterize two candidate cool galactic outflows in two relatively low mass, highly inclined Virgo cluster galaxies: NGC 4424 and NGC 4694. Previous analyses of observations using the Atacama Large Millimeter Array (ALMA) carbon monoxide (CO) line emission maps did not classify these sources as cool outflow hosts.  Using new high sensitivity, high spatial resolution, JWST mid-infrared photometry in the polycyclic aromatic hydrocarbon (PAH)-tracing F770W band, we identify extended structures present off of the stellar disk. The identified structures are bright in the MIRI F770W and F2100W bands, suggesting they include PAHs as well as other dust grains. As PAHs have been shown to be destroyed in hot, ionized gas, these structures are likely to be outflows of cool ( $T \leq 10^4$ K) gas. This work represents an exciting possibility for using mid infrared observations to identify and measure outflows in lower mass, lower star formation galaxies.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.12058"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

128  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
